In [19]:
# Import
import pandas as pd
from sqlalchemy import create_engine
from geopy import geocoders

database_path = "../da_job_data.sqlite"

In [2]:
# Create Engine
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [3]:
# Query All Records in the the Database
data = pd.read_sql("SELECT * FROM da_data", conn)

In [4]:
# Preview the Data
data.head()

,id,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,1,title,company_name,location,via,description,extensions,schedule_type,work_from_home,salary_hourly,salary_yearly,salary_standardized,description_tokens
1,2,Data Analyst,Applicantz,Anywhere,via LinkedIn,Open to work on W2 or on Corp-to-Corp (for vis...,"['41 minutes ago', 'Work from home', 'Contract...",Contractor,TRUE,,,,"['qlik', 'power_bi', 'power_bi']"
2,3,Data Analyst I (Remote Optional),Talentify.io,Anywhere,via LinkedIn,Talentify helps candidates around the world to...,"['15 hours ago', 'Work from home', 'Full-time']",Full-time,TRUE,,,,['excel']
3,4,Data Analyst II,EDWARD JONES,"Whiteman AFB, MO",via Monster,"At Edward Jones, we help clients achieve their...","['24 hours ago', 'Full-time', 'Health insurance']",Full-time,,,,,"['sas', 'snowflake', 'excel', 'power_bi', 'pyt..."
4,5,Data Analyst,Rose International,United States,via Indeed,Position Title\nPosition Number\nLocation...\n...,['7 hours ago'],,,,,,"['excel', 'tableau', 'cognos', 'sql', 'word']"


In [5]:
#assess data information and types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10498 entries, 0 to 10497
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   10498 non-null  int64 
 1   title                10498 non-null  object
 2   company_name         10498 non-null  object
 3   location             10498 non-null  object
 4   via                  10498 non-null  object
 5   description          10498 non-null  object
 6   extensions           10498 non-null  object
 7   schedule_type        10498 non-null  object
 8   work_from_home       10498 non-null  object
 9   salary_hourly        10498 non-null  object
 10  salary_yearly        10498 non-null  object
 11  salary_standardized  10498 non-null  object
 12  description_tokens   10498 non-null  object
dtypes: int64(1), object(12)
memory usage: 1.0+ MB


In [6]:
#drop the duplicate title row
data = data.drop([0])

In [7]:
#update salary_hourly, salary_yearly, and salary_standardized to be type FLOAT
#data['salary_hourly'] = data['salary_hourly'].astype(float)

#unable to convert due to empty rows

In [8]:
#create new dataframe for top locations of job postings
post_source_df = data['via'].value_counts().rename_axis('post_source').reset_index(name='count')
post_source_df.head(10)

,post_source,count
0,via LinkedIn,3996
1,via Upwork,1400
2,via ZipRecruiter,644
3,via Indeed,563
4,via BeBee,329
5,via Adzuna,292
6,via Trabajo.org,280
7,via Monster,251
8,via Snagajob,213
9,via My ArkLaMiss Jobs,204


In [9]:
#create new dataframe for the locations of the job postings and the count of appearances in the dataset
location_df = data['location'].value_counts(dropna=True).rename_axis('location').reset_index(name='count')
location_df

,location,count
0,Anywhere,4883
1,United States,2280
2,"Kansas City, MO",353
3,"Oklahoma City, OK",253
4,"Jefferson City, MO",207
...,...,...
336,"Shawnee, OK",1
337,"Shawnee County, KS",1
338,"Bismarck, AR",1
339,"Chickasha, OK",1


In [10]:
#filter out only locations with city and state provided for location mapping
clean_location_df = location_df[location_df['location'].str.contains(r'\w+,\s\w{2}')]
clean_location_df

,location,count
2,"Kansas City, MO",353
3,"Oklahoma City, OK",253
4,"Jefferson City, MO",207
5,"Tulsa, OK",129
6,"Columbia, MO",124
...,...,...
336,"Shawnee, OK",1
337,"Shawnee County, KS",1
338,"Bismarck, AR",1
339,"Chickasha, OK",1


In [20]:
gn = geocoders.GeoNames()

print(gn.geocode("Shawnee, OK"))

TypeError: __init__() missing 1 required positional argument: 'username'

In [11]:
print(clean_location_df.to_string())

                    location  count
2            Kansas City, MO    353
3          Oklahoma City, OK    253
4         Jefferson City, MO    207
5                  Tulsa, OK    129
6               Columbia, MO    124
7                Wichita, KS    108
8          Overland Park, KS     93
9            Bentonville, AR     90
10                Topeka, KS     82
12           Springfield, MO     65
13            Springdale, AR     55
14           Kansas City, KS     46
15          Lee's Summit, MO     34
16         San Francisco, CA     32
17          Fayetteville, AR     31
18           Warrensburg, MO     29
19                Lenexa, KS     27
20               Dearing, KS     26
21                Rogers, AR     22
22                  Enid, OK     22
23                 Maize, KS     19
24            Fort Smith, AR     19
25                Edmond, OK     18
26                 Rolla, MO     18
27                Olathe, KS     16
28            Stillwater, OK     15
29          Bartlesville, OK

In [12]:
#locations that were filtered out - for reference
filtered_location_df = location_df.drop(location_df[location_df['location'].str.contains(r'\w+,\s\w{2}')].index)
filtered_location_df

,location,count
0,Anywhere,4883
1,United States,2280
11,Missouri,72
32,Oklahoma,13
45,,11
59,Kansas,9
77,United States (+1 other),8
124,United States (+2 others),5
214,United States (+25 others),2
302,United States (+7 others),1


In [ ]:
# closing the database connection
conn.close()